# Code Deriving speed of Gravity and SME constraints

Author: Jay Tasson (jtasson@carleton.edu)

Date: version written on 171005 (yymmdd)

The code derives the speed of gravity constraints and constraints on the coefficients for Lorentz violation in the Standard-Model Extension that result from the 170817 GW-GRB observation.  The results appear in the [GW+GRB connection paper] : https://dcc.ligo.org/LIGO-P1700308.  Input values are the distance to the event and the observed time difference between the peak of the gravitational wave signal and the GRB observation, which appear in the [GW+GRB connection paper] and in addition for the SME constraints, the sky position of the event from [Coulter etal, MMA paper].

The references are:
<br>
[GW+GRB connection paper] : https://dcc.ligo.org/LIGO-P1700308
<br>
[LVC BNS discovery paper] : PRL, 119, 161101 (2017)  https://dcc.ligo.org/LIGO-P170817
<br>
[Coulter etal, MMA paper] : https://dcc.ligo.org/DocDB/0145/P1700294/002/multi-messenger-observations-v2.pdf page 5
[notes] : https://dcc.ligo.org/DocDB/0144/P1700245/004/notes.pdf


In [1]:
import sys
print(sys.version)

2.7.14 |Anaconda, Inc.| (default, Dec  7 2017, 17:05:42) 
[GCC 7.2.0]


In [2]:
#from lal import PC_SI, C_SI
import numpy as np
from scipy.special import sph_harm
import texttable as tt

PC_SI = 3.086e16 #parsec to meters conversion
C_SI = 299792458 #SI speed of lightdt

In [3]:
# time delay (in sec) between GW and gamma-ray signals at source according to astrophysical models. 
# first two entries are from conservative models; last two entries are from more exotic models 
dt_e_list = [0,10,-100,1000]       #[GW+GRB connection paper]

# observed parameters 
dt_o = 1.74        # observed time delay between GW and gamma-ray signal (sec) [GW+GRB connection paper]
# paper originally quoted 1.738 pm 0.048, this introduces no change
dt_error = [0.05,-0.05,0.05,-0.05]    # observed time error (sec) [GW+GRB connection paper]
dL = 26*1e6*PC_SI #90p lower luminosity dist (meters) [GW+GRB connection paper], [LVC BNS paper]

# existing upper limit on the isotrpic coefficient describing the difference 
# in Loretz violation in gravity and EM sectors. See, Table 1 of LIGO-P1700245-v3
k00_ub = 8e-5 

In [4]:
print('| dt_e (s) \t| dT (s) \t| dv  \t\t| k00_imprvmnt \t|')

# calculate the fractional difference (1-c_g/c) between the speed of GWs c_g and speed of light c
# for various assumed time delays at emission  
vlist = []
dverr = []
toterr = []
for j in range(0,len(dt_e_list)): 
        
    # difference between observed time delay and assumed time delay at emission
    dT = dt_o+dt_error[j]-dt_e_list[j]   
        
    # fractional difference between speed of GWs and light 
    dv = C_SI*dT/dL 
    vlist=np.append(vlist,dv)
    
    # calculate the improvement in the isotropic (j=m=0) coefficient describing the difference 
    # in Loretz violation in gravity and EM sectors. See, eq.(1) of LIGO-P1700245-v3
    k00 = np.sqrt(4*np.pi)*dv
    k00_improvement = k00_ub/k00 
        
    print ('| %.1e \t| %.1e \t| %.1e \t| %.0e \t|' %(dt_e_list[j], dT, dv, k00_improvement))


| dt_e (s) 	| dT (s) 	| dv  		| k00_imprvmnt 	|
| 0.0e+00 	| 1.8e+00 	| 6.7e-16 	| 3e+10 	|
| 1.0e+01 	| -8.3e+00 	| -3.1e-15 	| -7e+09 	|
| -1.0e+02 	| 1.0e+02 	| 3.8e-14 	| 6e+08 	|
| 1.0e+03 	| -1.0e+03 	| -3.7e-13 	| -6e+07 	|


In [5]:
# note form of spherical harmonics: sph_harm(m,j,ph,th)

# sky position from optical as reported in [Coulter etal, MMA paper]
rah = 13     #RA hours
ram = 9      #RA min
ras = 48.085 #RA sec

decd = -23     #dec degrees
decm = -22     #dec min
decs = -53.343 #dec sec

# convert to radians for spherical harmonics
ph = (rah+ram/60+ras/3600)*360/24*np.pi/180
th = (90-(decd+decm/60+decs/3600))*np.pi/180

# arrays to store bounds
upper=[]
lower=[]

d=4 #mass dimension

# loop over j and m of spherical harmonics
# This is based on the equation relating the fractional deviation in speeds 
# to the coefficients for Lorentz violation from [GW+GRB connection paper] 
# and elaborated on in the [notes]
for j in range(0,d-1):
    m=0 # for m=0, get both bounds on s with error
    upper=np.append(upper,vlist[0]/((-1)**j*.5*np.real(sph_harm(m,j,ph,th))))
    lower=np.append(lower,vlist[1]/((-1)**j*.5*np.real(sph_harm(m,j,ph,th))))
    for m in range(1,j+1):
        # for m>0, get both bounds on Im s and Re s with error
        upper=np.append(upper,vlist[0]/((-1)**j*np.real(sph_harm(m,j,ph,th))))
        lower=np.append(lower,vlist[1]/((-1)**j*np.real(sph_harm(m,j,ph,th))))
        upper=np.append(upper,-vlist[0]/((-1)**j*np.imag(sph_harm(m,j,ph,th))))
        lower=np.append(lower,-vlist[1]/((-1)**j*np.imag(sph_harm(m,j,ph,th))))
    
#adjust for table
upper=abs(upper)/1e-15
lower=-abs(lower)/1e-14
s = ['s_00','s_10','-Re s_11','Im s11','-s_20','-Re s_21','Im s_21','Re s_22','-Im s_22']

#create table
tab = tt.Texttable()
headings = ['lowerx10^-14','s','upperx10^-15']
tab.header(headings)

for row in zip(lower,s,upper):
    tab.add_row(row)

f=tab.draw()
print(f)

+--------------+----------+--------------+
| lowerx10^-14 |    s     | upperx10^-15 |
+==============+==========+==============+
| -2.201       | s_00     | 4.742        |
+--------------+----------+--------------+
| -3.123       | s_10     | 6.727        |
+--------------+----------+--------------+
| -1.020       | -Re s_11 | 2.196        |
+--------------+----------+--------------+
| -3.752       | Im s11   | 8.083        |
+--------------+----------+--------------+
| -3.913       | -s_20    | 8.430        |
+--------------+----------+--------------+
| -1.120       | -Re s_21 | 2.413        |
+--------------+----------+--------------+
| -4.123       | Im s_21  | 8.882        |
+--------------+----------+--------------+
| -1.117       | Re s_22  | 2.406        |
+--------------+----------+--------------+
| -1.904       | -Im s_22 | 4.101        |
+--------------+----------+--------------+


In [6]:
print vlist

[ 6.68812628e-16 -3.10493460e-15  3.80326466e-14 -3.73006891e-13]
